<a href="https://colab.research.google.com/github/jonclindaniel/LargeScaleComputing_A21/blob/main/in-class-activities/01_Introduction/1W_python_compilation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compiling Python with `numba` and `cython`

Reproduce Python function from lecture and measure its execution time:

In [ ]:
def loop(x, r):
    for i in range(r):
        x *= 2.5
    return x

%time loop(2, 10**6)

## Using `numba`

First, let's try compiling "Just in Time" using `numba`:

In [ ]:
from numba import jit

# jit compiles when we call the function for the first time
# nopython tries to run without involving Python interpreter
@jit(nopython=True)
def loop_jit(x, r):
  for i in range(r):
    x *= 2.5
  return x

%time loop_jit(2, 10**6) # includes compilation time

In [ ]:
%time loop_jit(2, 10**6) # much faster after compilation

In [ ]:
%timeit loop(2, 10**6) # better to time across multiple runs using `timeit`

In [ ]:
%timeit loop_jit(2, 10**6)

We might want to compile our code ahead of time, though, so that we can see a speed-up the first time we use it. `numba` allows us to compile ahead of time like so:


In [6]:
from numba.pycc import CC

# name of compiled module to create:
cc = CC('test_aot')

# name of function in module, with explicit data types required (4byte=32bit ints and floats)
@cc.export('loop_aot', 'f4(f4,i4)')
def loop_aot(x, r):
    for i in range(r):
        x *= 2.5
    return x

cc.compile()

Note that we now have a compiled object file (.so) in our current directory. This is a compiled module that contains our function.

In [ ]:
ls

To use our function, we just need to import our pre-compiled module, as we would any other Python module:

In [ ]:
import test_aot
%time test_aot.loop_aot(2, 10**6) # first time running it is fast this time

In [ ]:
%timeit test_aot.loop_aot(2, 10**6) # same overall performance as before

## Using `cython`

Another common way to compile Python code (albeit slightly uglier) is to compile our function via explicit `cython` static typing, like so (here, using the IPython `cython` extension to compile):

In [10]:
%load_ext cython

In [11]:
# will automatically convert Python -> C -> Compiled machine code
%%cython 

# explicitly add static types to function itself:
def loop_cython(float x, int r):
    cdef int i
    for i in range(r):
        x *= 2.5
    return x

In [ ]:
%timeit loop_cython(2, 10**6) # comparable performance to numba